In [1]:
import pandas as pd
import numpy as np

In [2]:
loans_df = pd.read_csv('data/concatenated.csv')
cash_df = pd.read_csv('data/cash.csv')

### Pivot Table

In [3]:
values_list = ['loan_part_amt',
              'loan_portion',
              'platform_weighted_net_aer',
              'platform_weighted_blended_ltv',
              'platform_weighted_term',
              'platform_weighted_ttm'
             ]
format_dict = { 'loan_part_amt' : '£{0:,.2f}', 'loan_portion' : '{:,.0f}', 'platform_weighted_blended_ltv' : '{:,.1%}', 'platform_weighted_net_aer' : '{:,.1%}', 'platform_weighted_term' : '{:,.0f}', 'platform_weighted_ttm' : '{:,.0f}' }
pd.pivot_table(loans_df.fillna(0), index='platform_account_name', values=values_list, aggfunc='sum', margins=True).style.format(format_dict)

,loan_part_amt,loan_portion,platform_weighted_blended_ltv,platform_weighted_net_aer,platform_weighted_term,platform_weighted_ttm
platform_account_name,,,,,,
Assetz Capital 90d Access,£911.34,270,62.0%,4.1%,7,0
CrowdProperty AutoInvest,"£2,000.00",31,65.9%,7.3%,13,6
Invest & Fund,"£2,500.00",14,63.4%,6.4%,12,5
Kuflink 12m Term 22043,"£1,000.02",120,65.3%,5.0%,12,2
Loanpad 60d access account,"£5,070.66",185,45.0%,5.4%,2,2
Proplend,"£3,000.00",3,47.1%,5.7%,22,13
All,"£14,482.02",623,348.7%,33.9%,67,28


### Sumary Table

In [4]:
portfolio_amt = loans_df['loan_part_amt'].sum()
cash_amt = cash_df['ib_cash'].sum()
total_amt = portfolio_amt + cash_amt # include interest bearing cash
no_loans = loans_df['loan_portion'].sum()
no_accounts = loans_df['platform_account_name'].nunique()

data = {
    'Date' : pd.to_datetime('today'),
    'Amount' : total_amt, 
    'Amount deployed' : portfolio_amt / total_amt,
    'No. loans' : no_loans,
    'Amount per loan' : portfolio_amt / no_loans,
    'No. accounts' : no_accounts,
    'Amount per account' : portfolio_amt / no_accounts,
    'Blended LTV' : [loans_df['portfolio_weighted_blended_ltv'].sum()],
    'Blended LTV (inc. cash)': [loans_df['portfolio_weighted_blended_ltv_inc_nonib_cash'].sum()],
    'Net AER' : [loans_df['portfolio_weighted_net_aer'].sum()],
    'Net AER (inc. cash)' : [loans_df['portfolio_weighted_net_aer_inc_nonib_cash'].sum()],
    'Avg. term' : loans_df['portfolio_weighted_term'].sum(),
    'Avg. time to maturity' : loans_df['portfolio_weighted_ttm'].sum()
}
df = pd.DataFrame.from_dict(data)

format_dict = {'Date': '{:%Y-%m}', 'Amount':'£{0:,.0f}', 'Amount deployed':'{:,.1%}', 'No. loans': '{0:,.0f}', 'Amount per loan':'£{0:,.2f}', 'Amount per account':'£{0:,.2f}', 'Blended LTV': '{:.1%}', 'Blended LTV (inc. cash)': '{:.1%}', 'Net AER': '{:.1%}', 'Net AER (inc. cash)': '{:.1%}', 'Avg. term' : '{:,.0f}', 'Avg. time to maturity' : '{:,.0f}' }
df.style.format(format_dict)
#weighted average time to maturity
#max time to maturity
#risk premiums over Gilts & FDs
#print in a snapshot table and export
#long and short defaults

,Date,Amount,Amount deployed,No. loans,Amount per loan,No. accounts,Amount per account,Blended LTV,Blended LTV (inc. cash),Net AER,Net AER (inc. cash),Avg. term,Avg. time to maturity
0,2023-06,"£14,571",99.4%,623,£23.25,6,"£2,413.67",53.9%,53.7%,5.8%,5.8%,10,5


In [5]:
pd.pivot_table(cash_df, index='platform_account_name', values=['ib_cash','nonib_cash'], aggfunc=np.sum, margins=True)

,ib_cash,nonib_cash
platform_account_name,,
Assetz Capital 90d Access,0.000000,0.00
CrowdProperty AutoInvest,0.000000,44.80
Invest & Fund,0.000000,5.58
Kuflink 12m Term 22043,0.000000,0.00
Loanpad 60d access account,89.338494,2.83
Proplend,0.000000,13.79
All,89.338494,67.00
